In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=7,
    mode='min',
    restore_best_weights=True
    )

# Optimasi

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.metrics import R2Score
import numpy as np
import tensorflow as tf

from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.basic import ArtificialBeeColonyAlgorithm

def get_hyperparameters(x):
    hidden_layers_choices = (1, 2, 3, 4, 5)
    nodes_choices = (8, 16, 32, 64, 128)
    activation_choices = ('relu', 'tanh', 'gelu')
    optimizer_choices = ('adam', 'sgd', 'rmsprop')

    hidden_layers = int(3 + x[0] * 2)
    nodes = []
    for i in range(6):
      temp = int(8 + x[i+1] * 120)
      nodes.append(temp)
    activation_layer = []
    for i in range(6):
      temp1 = activation_choices[int(x[i+7] * 2)]
      activation_layer.append(temp1)
    # activation = activation_choices[int(x[7] * 2)]
    optimizer = optimizer_choices[int(x[13] * 2)]
    params = {
        'hidden_layers': hidden_layers,
        'nodes': nodes,
        'activation': activation_layer,
        'optimizer': optimizer
    }
    return params

def get_model(x, input_shape):
    params = get_hyperparameters(x)

    model = Sequential()

    # Input layer
    model.add(Input(shape=input_shape))
    model.add(Dense(params['nodes'][0],
                   activation=params['activation'][0]))

    # Hidden layers
    for _ in range(params['hidden_layers'] - 1):
        model.add(Dense(params['nodes'][_+1],
                       activation=params['activation'][_+1]))

    # Output layer
    model.add(Dense(1, activation='linear'))  # untuk regresi

    # Set optimizer
    if params['optimizer'] == 'adam':
        opt = Adam()
    elif params['optimizer'] == 'sgd':
        opt = SGD()
    else:
        opt = RMSprop()

    # Compile model
    model.compile(optimizer=opt, loss='mse', metrics=[R2Score()])

    return model

best_val_loss = np.inf

class ANNHyperparameterOptimization(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=14, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        global best_val_loss
    # Buat model dengan parameter dari solusi x
        model = get_model(x, input_shape=(self.X_train.shape[1],))

        
            # Train model
        history = model.fit(
            self.X_train, self.y_train,
            epochs=100,
            batch_size=32,
            validation_split=0.2,
            verbose=0,
            callbacks=[early_stopping]
        )

            # Gunakan R² validation terakhir sebagai fitness
            # Karena kita ingin memaksimalkan, kita kembalikan nilai positif
        val_loss = history.history['val_loss'][-1]

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            df_history = pd.DataFrame(history.history)
            df_history.to_csv('training_history10n.csv', index=False)
            model.save('best_model10n.h5')
        
        return val_loss

# Inisialisasi problem
problem = ANNHyperparameterOptimization(X_train, y_train)

# Setup task untuk optimasi (minimasi MSE)
task = Task(problem, max_iters=20, optimization_type=OptimizationType.MINIMIZATION)

# Inisialisasi algoritma ABC
algorithm = ArtificialBeeColonyAlgorithm(
    population_size=10
)

# Jalankan optimasi
best_params, best_fitness = algorithm.run(task)

konvergen_data = task.convergence_data()
df_konvergen = pd.DataFrame(konvergen_data)
df_konvergen.to_csv('konvergen_data10n.csv', index=False)

# Dapatkan parameter terbaik
best_hyperparameters = get_hyperparameters(best_params)
print('Parameter terbaik:', best_hyperparameters)


Parameter terbaik: {'hidden_layers': 4, 'nodes': [11, 53, 90, 112, 104, 116], 'activation': ['tanh', 'tanh', 'relu', 'relu', 'tanh', 'tanh'], 'optimizer': 'adam'}


In [5]:
# best_model.save('best_model50nn.h5')

In [6]:
print(vars(algorithm))

{'population_size': 10, 'initialization_function': <function default_individual_init at 0x0000018A5147AA20>, 'individual_type': <class 'niapy.algorithms.basic.abc.SolutionABC'>, 'callbacks': <niapy.callbacks.CallbackList object at 0x0000018A26288380>, 'rng': Generator(PCG64) at 0x18A512D6EA0, 'exception': None, 'limit': 100, 'food_number': 5}
